In [610]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.core import Dropout
from keras.regularizers import Regularizer
import numpy as np
import pandas as pd
from numpy.random import seed
from tensorflow import set_random_seed

In [611]:
seed(1)
set_random_seed(1)

Read the dataset and separate into data and labels.

In [612]:
bioresponse_train = pd.read_csv('bioresponse_dataset/train.csv')

In [613]:
bioresponse_train_label = bioresponse_train['Activity']
bioresponse_train_data = bioresponse_train.iloc[:, 1:]

Get number of features for the input of Neural Network.

In [631]:
num_features = bioresponse_train_data.shape[1]
num_instances = bioresponse_train_data.shape[0]
print("Number of features:", num_features)
print("Number of instances:", num_instances)

Number of features: 1776
Number of instances: 3751


Fix the number of the first layer outputs, dropout rate for regulirization, optimizer, loss function, metrics, batch_size, epochs and split.

In [773]:
first_layer_size = 32
N = 4
dropout_ratio = 0.25
optimizer = 'adam'
loss = 'binary_crossentropy'
metrics = ['accuracy']
batch_size = 32
epochs = 5
split = 0.2

Initialize Neural Network with two hidden layers. First contains *first_layer_size* neurons, second *N* times less.

In [774]:
model = Sequential([
    Dense(first_layer_size, input_dim=num_features),
    Activation('relu'),
    Dropout(dropout_ratio),
    Dense(int(first_layer_size / N)),
    Activation('relu'),
    Dropout(dropout_ratio),
    Dense(1),
    Activation('sigmoid')
])

Compile model using optimizer. loss function and metrics determined before.

In [766]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [767]:
model.fit(bioresponse_train_data, bioresponse_train_label, validation_split=split,
          batch_size=batch_size, epochs=epochs)

Train on 3000 samples, validate on 751 samples
Epoch 1/5
3000/3000 [==============================] - 7s 2ms/step - loss: 0.6630 - acc: 0.5860 - val_loss: 0.6118 - val_acc: 0.7124
Epoch 2/5
3000/3000 [==============================] - 1s 348us/step - loss: 0.5792 - acc: 0.7067 - val_loss: 0.5299 - val_acc: 0.7443
Epoch 3/5
3000/3000 [==============================] - 1s 341us/step - loss: 0.5187 - acc: 0.7510 - val_loss: 0.5203 - val_acc: 0.7670
Epoch 4/5
3000/3000 [==============================] - 1s 345us/step - loss: 0.4924 - acc: 0.7720 - val_loss: 0.4919 - val_acc: 0.7803
Epoch 5/5
3000/3000 [==============================] - 1s 345us/step - loss: 0.4759 - acc: 0.7853 - val_loss: 0.4987 - val_acc: 0.7656


Make predictions for testing data.

In [768]:
bioresponse_test_data = pd.read_csv('bioresponse_dataset/test.csv')

In [769]:
predicted = pd.DataFrame(model.predict_proba(bioresponse_test_data))
predicted.head()

,0
0,0.880332
1,0.932615
2,0.287539
3,0.909401
4,0.071823


In [770]:
predicted.index += 1
predicted.head()

,0
1,0.880332
2,0.932615
3,0.287539
4,0.909401
5,0.071823


In [771]:
predicted.to_csv('./bioresponse_dataset/keras_predicted.csv', header=['PredictedProbability'], index=True, index_label = ['MoleculeId'])

Logloss value returned by kaggle for such architecture is 0.5, worse than xgboost and random forest classefier.